In [1]:
import numpy as np
from scipy.stats import t

## а) Определить влияние возраста на содержание иммуноглобулина

In [42]:
Y = np.array([
    83, 85,
    84, 85, 85, 86, 86, 87,
    86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90,
    89, 90, 90, 91,
    90, 92
])
x1 = np.tile(np.array([1, 0, 0, 0, 0]), (2, 1))
x2 = np.tile(np.array([0, 1, 0, 0, 0]), (6, 1))
x3 = np.tile(np.array([0, 0, 1, 0, 0]), (11, 1))
x4 = np.tile(np.array([0, 0, 0, 1, 0]), (4, 1))
x5 = np.tile(np.array([0, 0, 0, 0, 1]), (2, 1))
PSI = np.vstack((x1, x2, x3, x4, x5))

F = PSI.T @ PSI
F_inv = np.linalg.inv(F)

beta_ = F_inv @ PSI.T @ Y

e = Y - PSI@beta_
RSS = e.T @ e
TSS = np.sum(((Y - Y.mean()).ravel()) ** 2)
R_2 = (TSS - RSS) / TSS
print(f"Коэффициент детерминации = {np.round(R_2, 4)}  >  0.7  => Влияние возраста существенно")

Коэффициент детерминации = 0.8106  >  0.7  => Влияние возраста существенно


In [23]:
n = np.size(Y)
p = np.size(beta_)
alpha = 0.05

## б) Провести попарное сравнение средних в рамках регрессионной модели с учетом множественности проверяемых гипотез

In [29]:
p_values = {}
for i, beta_1 in enumerate(beta_wave[:-1]):
    for j, beta_2 in enumerate(beta_wave[i+1:]):
        delta_wave = abs((beta_1-beta_2)*(n-p)**0.5 / (RSS*(F_inv[i, i] + F_inv[j, j]))**0.5)
        p_value = t(n-p).sf(delta_wave) + t(n-p).cdf(-delta_wave)
        p_values[(i, i+j+1)] = [p_value, delta_wave]
p_values

{(0, 1): [0.1784265447269502, 1.394629046312061],
 (0, 2): [0.00031201035098750375, 4.3478012377276585],
 (0, 3): [5.078817383805672e-07, 7.2570206974386355],
 (0, 4): [3.0205166476170114e-07, 7.515101582874102],
 (1, 2): [0.015713161592785407, 2.6397364657632196],
 (1, 3): [5.186378641056689e-07, 7.246705097771455],
 (1, 4): [2.786279378963573e-09, 10.07575604197543],
 (2, 3): [0.01574118243227401, 2.638916617250419],
 (2, 4): [1.0562432883518134e-05, 5.828949776349431],
 (3, 4): [0.2957913522747472, 1.073585940410586]}

In [41]:
p_values_incresing = sorted(p_values.items(), key=lambda item: item[1][0])
for index, item in enumerate(p_values_incresing):
    key, value = item
    i, j = key
    p_value, delta_wave = value
    alpha_tmp = alpha/(len(results)-index) #метод холма-бонферрони
    print(f'beta_{i+1} == beta_{j+1}: p-value(delta > {np.round(delta_wave, 4)} | H_0) =' +\
          f' {np.round(p_value, 10)} ' +\
          f'{">" if p_value > alpha_tmp else "<"} alpha = {np.round(alpha_tmp, 4)} --> Коэффициенты ' +\
          f' {"равны!" if p_value > alpha_tmp else "неравны!"}')

beta_2 == beta_5: p-value(delta > 10.0758 | H_0) = 2.8e-09 < alpha = 0.005 --> Коэффициенты  неравны!
beta_1 == beta_5: p-value(delta > 7.5151 | H_0) = 3.021e-07 < alpha = 0.0056 --> Коэффициенты  неравны!
beta_1 == beta_4: p-value(delta > 7.257 | H_0) = 5.079e-07 < alpha = 0.0062 --> Коэффициенты  неравны!
beta_2 == beta_4: p-value(delta > 7.2467 | H_0) = 5.186e-07 < alpha = 0.0071 --> Коэффициенты  неравны!
beta_3 == beta_5: p-value(delta > 5.8289 | H_0) = 1.05624e-05 < alpha = 0.0083 --> Коэффициенты  неравны!
beta_1 == beta_3: p-value(delta > 4.3478 | H_0) = 0.0003120104 < alpha = 0.01 --> Коэффициенты  неравны!
beta_2 == beta_3: p-value(delta > 2.6397 | H_0) = 0.0157131616 > alpha = 0.0125 --> Коэффициенты  равны!
beta_3 == beta_4: p-value(delta > 2.6389 | H_0) = 0.0157411824 < alpha = 0.0167 --> Коэффициенты  неравны!
beta_1 == beta_2: p-value(delta > 1.3946 | H_0) = 0.1784265447 > alpha = 0.025 --> Коэффициенты  равны!
beta_4 == beta_5: p-value(delta > 1.0736 | H_0) = 0.29579135